In [1]:
class Bound:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return f"({self.x}, {self.y})"

    def __contains__(self, item):
        i, j = item
        return 0 <= i < self.x and 0 <= j < self.y

    def __iter__(self):
        for i in range(self.x):
            for j in range(self.y):
                yield (i, j)
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def limit(self, item):
        i, j = item
        return max(0, min(i, self.x - 1)), max(0, min(j, self.y - 1))

In [2]:
# Each octopus has an energy level - your submarine can remotely measure the energy level of each octopus (your puzzle input). For example:
# The energy level of each octopus is a value between 0 and 9. Here, the top-left octopus has an energy level of 5, the bottom-right one has an energy level of 6, and so on.

# You can model the energy levels and flashes of light in steps. During a single step, the following occurs:

# First, the energy level of each octopus increases by 1.
# Then, any octopus with an energy level greater than 9 flashes. This increases the energy level of all adjacent octopuses by 1, including octopuses that are diagonally adjacent. If this causes an octopus to have an energy level greater than 9, it also flashes. This process continues as long as new octopuses keep having their energy level increased beyond 9. (An octopus can only flash at most once per step.)
# Finally, any octopus that flashed during this step has its energy level set to 0, as it used all of its energy to flash.
# Adjacent flashes can cause an octopus to flash on a step even if it begins that step with very little energy. Consider the middle octopus with 1 energy in this situation:

import numpy as np

neighbors = np.array([
    [-1, -1], [-1, 0], [-1, 1],
    [ 0, -1],          [ 0, 1],
    [ 1, -1], [ 1, 0], [ 1, 1]
])

class Board:
    def __init__(self, board):
        self.board = np.array(board)
        self.bound = Bound(*self.board.shape)

    def step(self):
        self.board += 1
        self.flashed = np.zeros_like(self.board, dtype=bool)

        while True:
            new_flashes = 0
            
            conditions = (self.board > 9) & ~self.flashed
            for i, j in zip(*np.where(conditions)):
                if self.flashed[i, j]:
                    continue
                
                new_flashes += 1
                self.flashed[i, j] = True
                pos = np.array([i, j])
                for neighbor in neighbors + pos:
                    if neighbor in self.bound:
                        self.board[tuple(neighbor)] += 1
            
            if not new_flashes:
                break
        
        self.board[self.flashed] = 0
        return self.flashed.sum()

    def __str__(self):
        return "\n".join(
            "".join(f"{x} " if x < 10 else "* " for x in line)
            for line in self.board
        )


In [13]:
with open("input11.txt") as f:
    board = [
        [int(x) for x in line]
        for line in f.read().splitlines()
    ]

board = Board(board)

In [14]:
step = 0

while np.any(board.board):
    board.step()
    step += 1

step

237